In [3]:
#import all libs
import tweepy
import json
import requests
from nltk.corpus import stopwords # stopword examples, 'its', 'on', 'the', etc <---- will be helpful later
# most pythonistas will rename pandas as pd, numpy as np, and datetime as dt for short (you don't have to)
import pandas as pd
import numpy as np
import datetime as dt

In [4]:
# open file with keys and set the path to your credentials JSON file
# you'll need to replace my file with yours
credentials = "keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [5]:
bearer_token = api_tokens["bearer_token"]

In [6]:
# we only want status info so we won't need the higher level access
client = tweepy.Client(bearer_token=bearer_token)

In [7]:
#instead of searching for most recent tweet could instead search for a specific date/time frame. 
tweets_bakeoff = client.search_recent_tweets(
    query = "#GBBO -is:retweet",  # searches for #GBBO while ignoring retweets
    max_results = 100
)

tweets_bakeoff.data

[<Tweet id=1600492157663592449 text='Matt Hancock announces plans to quit Parliament the day after Matt Lucas leaves a vacancy for a presenter on #gbbo...'>,
 <Tweet id=1600491438046846977 text="@realTuckFrumper #GBBO doesn't air in US , sadly.">,
 <Tweet id=1600490850424717313 text='Finally some good news! #GBBO  https://t.co/4zHqrAx4Mn'>,
 <Tweet id=1600490348190613508 text='Just another example of how awesome our Research Team is @YorkResearch! Big love to CG1 Research Nurse Heidi R for keeping us all sweet today with her GBBO sweepstake winners bake!🏆🧁💙#lifeissweet #ResearchNursing #BeKind @YSTeachingNHS #GBBO https://t.co/hYz2otjsxJ'>,
 <Tweet id=1600488405053444096 text='Now we know who’s replacing Matt Lucas on #GBBO, then. https://t.co/KWF6rK9MI6'>,
 <Tweet id=1600488045232459776 text='Matt is passing the "baguette" on 🥖 #GBBO https://t.co/9MgqVrowDl'>,
 <Tweet id=1600484964642033665 text='Is Matt Lucas jumping ship at the right time? This year’s Bake Off had lost its magic for

In [8]:
#make tweets into a list
bakeoff_tweets = []
for tweet in tweets_bakeoff.data:
    bakeoff_tweets.append(tweet.text)
    
bakeoff_tweets

['Matt Hancock announces plans to quit Parliament the day after Matt Lucas leaves a vacancy for a presenter on #gbbo...',
 "@realTuckFrumper #GBBO doesn't air in US , sadly.",
 'Finally some good news! #GBBO  https://t.co/4zHqrAx4Mn',
 'Just another example of how awesome our Research Team is @YorkResearch! Big love to CG1 Research Nurse Heidi R for keeping us all sweet today with her GBBO sweepstake winners bake!🏆🧁💙#lifeissweet #ResearchNursing #BeKind @YSTeachingNHS #GBBO https://t.co/hYz2otjsxJ',
 'Now we know who’s replacing Matt Lucas on #GBBO, then. https://t.co/KWF6rK9MI6',
 'Matt is passing the "baguette" on 🥖 #GBBO https://t.co/9MgqVrowDl',
 'Is Matt Lucas jumping ship at the right time? This year’s Bake Off had lost its magic for me https://t.co/pPnqEK2a2j #gbbo',
 "That's quite the list. #GBBO https://t.co/S4Mk8K1xhD",
 'Matt Lucas quits Bake Off. https://t.co/IZ6Zrrt7eA #GBBO',
 'Matt Lucas quits Bake Off. https://t.co/ZuXmR6V8df #GBBO',
 'Is #MattLucas really too busy or h

In [9]:
tweets_Euston = client.search_recent_tweets(
    query = "Euston -is:retweet",  # searches for Euston while ignoring retweets
    max_results = 100
)

tweets_Euston.data

[<Tweet id=1600492285443153921 text='@AvantiWestCoast @LNRailway @CrossCountryUK @GWRHelp Hi, I have a ticket from Euston to Macclesfield, will @crosscountryuk let me use my Avanti ticket to get all the way home or do I get kicked off at New Street? Thanks!'>,
 <Tweet id=1600491966411804674 text='@AvantiWestCoast your train manager on the 12:47 from Liverpool Lime St to Euston is the best.'>,
 <Tweet id=1600491925290864642 text='https://t.co/tiS6F3eQMX what next you going to throw at us British Rail?'>,
 <Tweet id=1600491902863872000 text="@AvantiWestCoast the 12.03 MK to Euston is now moving forward - just at King's Langley. Is there an eta for our arrival at Euston? Thank you.">,
 <Tweet id=1600491713373650945 text='@AvantiWestCoast @LNRailway @CrossCountryUK @GWRHelp Hello Avanti, I am trying to get to Lancaster from London-Euston - what would you recommend? Going via East coast, will take me too far East. Thanks!'>,
 <Tweet id=1600491375543427074 text='@AvantiWestCoast hi, can we u

In [10]:
Euston_tweets = []
for tweet in tweets_Euston.data:
    Euston_tweets.append(tweet.text)
    
Euston_tweets

['@AvantiWestCoast @LNRailway @CrossCountryUK @GWRHelp Hi, I have a ticket from Euston to Macclesfield, will @crosscountryuk let me use my Avanti ticket to get all the way home or do I get kicked off at New Street? Thanks!',
 '@AvantiWestCoast your train manager on the 12:47 from Liverpool Lime St to Euston is the best.',
 'https://t.co/tiS6F3eQMX what next you going to throw at us British Rail?',
 "@AvantiWestCoast the 12.03 MK to Euston is now moving forward - just at King's Langley. Is there an eta for our arrival at Euston? Thank you.",
 '@AvantiWestCoast @LNRailway @CrossCountryUK @GWRHelp Hello Avanti, I am trying to get to Lancaster from London-Euston - what would you recommend? Going via East coast, will take me too far East. Thanks!',
 '@AvantiWestCoast hi, can we use your service from Euston to Coventry when it’s back up and running with a London Midland train ticket due to delays?',
 'Euston Station travel chaos as firefighters free trapped commuters ... (https://t.co/gQbuTU

In [35]:
import nltk
from nltk.corpus import stopwords
#create function
def get_features(text):
    # create array of keywords
    #stopwords are lowercase
    text = text.lower()
    #split the test into an array of words
    words_list = text.split()
    #this isn't necessary but will make the next part shorter
    s_words = stopwords.words('english')
    
    #keywords = dict([(w, True) for w in words if w not in s_words and not w.isdigit() and '/' not in w])
    
    #return keywords
    #extend s_words to remove a number of annoying characters
    keywords = [term for term in words_list if term not in s_words and '@' not in term]
    while len(keywords) < 3:
        keywords.append('None')
    return {'first_keyword': keywords[0], 'second_keyword': keywords[1], 'third_keyword': keywords[2]}

get_features(Euston_tweets[50])
#collections library, counter func could give you the most used words

{'first_keyword': 'london',
 'second_keyword': 'euston',
 'third_keyword': 'station'}

In [12]:


#label_features = {'#GBBO': [], 'Euston': []}
#for tweet in bakeoff_tweets:
    #features = get_features(tweet)
    #label_features['#GBBO'].append(features)
    
    
#create a list of tuples containing the featureet and the label
#features and label list will contain my features and my label

features_and_labels = []
for tweet in bakeoff_tweets:
    features_and_labels.append((get_features(tweet), '#GBBO'))

In [13]:
for tweet in Euston_tweets:
    #features = get_features(tweet)
    #label_features['Euston'].append(features)
    features_and_labels.append((get_features(tweet), 'Euston'))

In [14]:
features_and_labels

[({'first_keyword': 'matt',
   'second_keyword': 'hancock',
   'third_keyword': 'announces'},
  '#GBBO'),
 ({'first_keyword': '#gbbo', 'second_keyword': 'air', 'third_keyword': 'us'},
  '#GBBO'),
 ({'first_keyword': 'finally',
   'second_keyword': 'good',
   'third_keyword': 'news!'},
  '#GBBO'),
 ({'first_keyword': 'another',
   'second_keyword': 'example',
   'third_keyword': 'awesome'},
  '#GBBO'),
 ({'first_keyword': 'know',
   'second_keyword': 'who’s',
   'third_keyword': 'replacing'},
  '#GBBO'),
 ({'first_keyword': 'matt',
   'second_keyword': 'passing',
   'third_keyword': '"baguette"'},
  '#GBBO'),
 ({'first_keyword': 'matt',
   'second_keyword': 'lucas',
   'third_keyword': 'jumping'},
  '#GBBO'),
 ({'first_keyword': "that's",
   'second_keyword': 'quite',
   'third_keyword': 'list.'},
  '#GBBO'),
 ({'first_keyword': 'matt',
   'second_keyword': 'lucas',
   'third_keyword': 'quits'},
  '#GBBO'),
 ({'first_keyword': 'matt',
   'second_keyword': 'lucas',
   'third_keyword': 'q

In [15]:
#shuffle the labels
import random
random.shuffle(features_and_labels)

In [16]:
features_and_labels

[({'first_keyword': 'matt,',
   'second_keyword': 'leaving',
   'third_keyword': '#gbbo'},
  '#GBBO'),
 ({'first_keyword': 'win',
   'second_keyword': 'champagne',
   'third_keyword': 'afternoon'},
  '#GBBO'),
 ({'first_keyword': 'acceptance',
   'second_keyword': 'place',
   'third_keyword': 'allow'},
  'Euston'),
 ({'first_keyword': 'matt',
   'second_keyword': 'lucas',
   'third_keyword': 'quits'},
  '#GBBO'),
 ({'first_keyword': '12.03',
   'second_keyword': 'mk',
   'third_keyword': 'euston'},
  'Euston'),
 ({'first_keyword': 'trains',
   'second_keyword': 'lime',
   'third_keyword': 'street'},
  'Euston'),
 ({'first_keyword': 'never',
   'second_keyword': 'right',
   'third_keyword': 'show.'},
  '#GBBO'),
 ({'first_keyword': 'found',
   'second_keyword': '#mattlucas',
   'third_keyword': 'leaving'},
  '#GBBO'),
 ({'first_keyword': 'every',
   'second_keyword': 'episode',
   'third_keyword': '#gbbo'},
  '#GBBO'),
 ({'first_keyword': 'hello',
   'second_keyword': 'avanti,',
   'thi

In [17]:
#split 80:20 training data and testing data
import math
#get how many elements in my list is 80 percent
split_num = math.floor(len(features_and_labels) * .8)
split_num

159

In [18]:
#80% of my data
training_set = features_and_labels[:split_num]
#20% of my data
testing_set = features_and_labels[split_num:]

In [19]:
import nltk
#create our classifier
#baseline is 50% because I have two labels/outcomes
classifier = nltk.NaiveBayesClassifier.train(training_set)

In [20]:
classifier.classify(get_features("As Matt Lucas is leaving Bake Off, I cannot say this enough, bring in a lesbian who can present #GBBO properly."))

'#GBBO'

In [21]:
#test accuracy
nltk.classify.accuracy(classifier, testing_set)

0.85

In [22]:
classifier.show_most_informative_features(10)

Most Informative Features
           first_keyword = 'absolutely'    #GBBO : Euston =      1.1 : 1.0
           first_keyword = 'many'          #GBBO : Euston =      1.1 : 1.0
           first_keyword = "we're"         #GBBO : Euston =      1.1 : 1.0
           third_keyword = 'back'          #GBBO : Euston =      1.1 : 1.0
          second_keyword = 'really'        #GBBO : Euston =      1.1 : 1.0


In [31]:
#new function to improve accuaracy , set features
def set_features(text):
    # create array of keywords
    #stopwords are lowercase
    text = text.lower()
    #split the test into an array of words
    words_list = text.split()
    #this isn't necessary but will make the next part shorter
    s_words = stopwords.words('english')
    s_words.append('many')  #need to add the words I want removed to stopwords somehow, this hasn't worked. 
    #keywords = dict([(w, True) for w in words if w not in s_words and not w.isdigit() and '/' not in w])
    
    #return keywords
    char_i_hate = ['@']
    
    keywords = [term for term in words_list if term not in s_words if term not in s_words and not any(c in term for c in char_i_hate)] 
    while len(keywords) < 3:
        keywords.append('None')
    return {'first_keyword': keywords[0], 'second_keyword': keywords[1], 'third_keyword': keywords[2]}


In [32]:
classifier.classify(set_features("As Matt Lucas is leaving Bake Off, I cannot say this enough, bring in a lesbian who can present #GBBO properly."))

'#GBBO'

In [36]:
#test accuracy
nltk.classify.accuracy(classifier, testing_set)

0.85